<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание данных</a></span></li><li><span><a href="#Откроем-и-изучим-файл" data-toc-modified-id="Откроем-и-изучим-файл-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Откроем и изучим файл</a></span></li><li><span><a href="#Разобьем-данные-на-выборки" data-toc-modified-id="Разобьем-данные-на-выборки-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Разобьем данные на выборки</a></span></li><li><span><a href="#Исследуем-модели" data-toc-modified-id="Исследуем-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Исследуем модели</a></span></li><li><span><a href="#Проверим-модель-на-тестовой-выборке" data-toc-modified-id="Проверим-модель-на-тестовой-выборке-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Проверим модель на тестовой выборке</a></span></li><li><span><a href="#Проверим-модели-на-адекватность" data-toc-modified-id="Проверим-модели-на-адекватность-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Проверим модели на адекватность</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:"><span class="toc-item-num">7&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></div>

# Рекомендация тарифов

У нас есть данные о поведении клиентов, которые перешли на рекомендованные ранее тарифы (проект 3). Построим модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных проведена в предыдущем проекте (3).

Нужно получить модель с максимально большим значением *accuracy*, по крайней мере от 0.75.

## Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.  
Известно:  
`сalls` — количество звонков,  
`minutes` — суммарная длительность звонков в минутах,  
`messages` — количество sms-сообщений,  
`mb_used` — израсходованный интернет-трафик в Мб,  
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0)  

## Откроем и изучим файл

In [1]:

import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


 - Все значения заполнены, столбцы названы правильно, расход памяти ничтожный.

## Разобьем данные на выборки

 - Выделим целевой признак.

In [6]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

 - Разобьем выборки на 3 части в соотношении 3:1:1 для тренировочного набора данных, валидационного и тестового соответственно. 

In [7]:
features_train, features_valid, target_train, target_valid =\
train_test_split(features, target, test_size=0.2, random_state=12345)

features_train, features_test, target_train, target_test =\
train_test_split(features_train, target_train, test_size=0.25, random_state=12345) 
# 0.25 берется от 0.8 и это = 0.2 от исходной выборки.

## Исследуем модели

 - Проверим дерево решений и подберем лучшие параметры для него.

In [8]:

best_model = None
best_result = 0
best_depth = 0
for depth in range(1,11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
print("Accuracy лучшей модели дерева решений на валидационной выборке:", best_result, ", глубина:", best_depth)


Accuracy лучшей модели дерева решений на валидационной выборке: 0.7884914463452566 , глубина: 5


 - Проверим случайный лес.

In [9]:

best_model = None
best_result = 0
best_depth = 0
best_n_est = 0
for depth in range(1,11):
    for est in range(10, 50, 10):
        model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions)
        if result > best_result:
            best_model = model
            best_result = result
            best_depth = depth
            best_n_est = est
print("Accuracy лучшей модели случайного леса на валидационной выборке:", best_result, ", глубина:", best_depth, ", кол-во деревьев:", best_n_est)


Accuracy лучшей модели случайного леса на валидационной выборке: 0.8009331259720062 , глубина: 6 , кол-во деревьев: 20


 - Теперь логистическая регрессия.

In [10]:

model = LogisticRegression(random_state=12345, solver='lbfgs')
model.fit(features_train, target_train)
features = model.predict(features_valid) 
result = model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)


Accuracy модели логистической регрессии на валидационной выборке: 0.7589424572317263


 - Случайный лес показал себя лучше всех.

## Проверим модель на тестовой выборке

 - Теперь модели с лучшими параметрами проверим на тестовой выборке.
 - Тренировать будем на объедененных выборках `train` и `valid`.

In [11]:

features_combine = pd.concat([features_train, features_valid], axis=0, ignore_index=True)
target_combine = pd.concat([target_train, target_valid], axis=0, ignore_index=True)


In [12]:

model = DecisionTreeClassifier(random_state=12345, max_depth=5)
model.fit(features_combine, target_combine)
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print("Accuracy лучшей модели дерева решений на тестовой выборке:", result)


Accuracy лучшей модели дерева решений на тестовой выборке: 0.7791601866251944


In [13]:

model = RandomForestClassifier(random_state=12345, max_depth=6, n_estimators=20)
model.fit(features_combine, target_combine)
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print("Accuracy лучшей модели случайного леса на тестовой выборке:", result)


Accuracy лучшей модели случайного леса на тестовой выборке: 0.7807153965785381


In [21]:

model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=110)
model.fit(features_combine, target_combine)
predictions = model.predict(features_test)
result = accuracy_score(target_test, predictions)
print("Accuracy модели логистической регрессии на тестовой выборке:", result)


Accuracy модели логистической регрессии на тестовой выборке: 0.7216174183514774


 - Снова лес впереди.                                       *(0.781)*  
 - Если нужна скорость, то на втором месте дерево решений.  *(0.779)*  
 - Логистическая регрессия показала себя хуже всех.         *(0.722)*  

## Проверим модели на адекватность

 - Проверим адекватность модели сравнив целевой признак с целевым признаком выбранным случайным образом.

In [15]:

np.random.seed(0)
df['random_is_ultra'] = np.random.choice([0, 1], len(df))
df


,calls,minutes,messages,mb_used,is_ultra,random_is_ultra
0,40.0,311.90,83.0,19915.42,0,0
1,85.0,516.75,56.0,22696.96,0,1
2,77.0,467.66,86.0,21060.45,0,1
3,106.0,745.53,81.0,8437.39,1,0
4,66.0,418.74,1.0,14502.75,0,1
...,...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1,1
3210,25.0,190.36,0.0,3275.61,0,0
3211,97.0,634.44,70.0,13974.06,0,0
3212,64.0,462.32,90.0,31239.78,0,1


In [16]:

result = accuracy_score(df['is_ultra'], df['random_is_ultra'])
print("Accuracy модели логистической регрессии на тестовой выборке:", result)


Accuracy модели логистической регрессии на тестовой выборке: 0.4940883634100809


## Вывод:

Наши модели явно предсказывают целевой признак (тариф) лучше случайности.  
**Случайный лес показал себя наиболее точным.**    
**На втором месте по точности, но быстрее в работе дерево решений.**    
**Логистическая регрессия показала себя хуже всех.**  